In [19]:
import numpy as np
import pandas as pd
from scipy import sparse
import pickle


def recommend():
    # Import user games, steam game data and other user data
    myg = pd.read_excel("../01_Raw Data/Data Files/mygames.xlsx")
    sg = pd.read_csv("../02_Data Cleaning/Cleaned Data/clean_sg.csv", dtype={"appid":"str"})
    # Load from sparse file and column headers
    ug = sparse.load_npz("Data/User_Data.npz")
    with open ("../02_Data Cleaning/Cleaned Data/g_list.txt", 'rb') as fp:
        mygam = pickle.load(fp)
     # Create dataframe with user games in
    mygames = pd.DataFrame(index=mygam)
    mygames["mg"] = [(int(col) in list(myg.appid))*1 for col in mygam]
    nmg = sum(mygames["mg"])
    # Convert dataframe to sparse matrix
    mg = sparse.coo_matrix(mygames.astype("int32").values).tocsr()
    # Dot product of matricies to count number of games both parties own
    prod = np.array(ug.dot(mg).sum(1))
    # Count number of games a user has
    ct = np.array(ug.sum(1))
    # Calc similarity score
    sim = prod**2 / ((nmg+1)*(ct+1))
    # Extract indicies of 10 best matches. 
    tt = pd.DataFrame(sim).nlargest(30, 0).sort_index()
    # Weight scores in matrix by similarity and then sum to get total score by game
    s = ug[tt.index].multiply(np.array([[i] for i in tt[0]])).sum(0)
    mygames["scores"] = s.tolist()[0]
    return mygames.join(sg.set_index("appid"))


otpt = recommend()
otpt.nlargest(10,"scores")

,mg,scores,name,required_age,price,initialprice,discount,year,positive,negative,...,median_forever,median_2weeks,ccu,genre,tags,achievements,developers,publishers,platforms,languages
220,0,2.519678,Half-Life 2,0,1.99,9.99,80,2004,100430,2940,...,349,144,1586,Action,"{'FPS': 3711, 'Action': 2616, 'Sci-fi': 2275, ...",33,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}","English, French, German, Italian, Korean, Span..."
320,0,2.519678,Half-Life 2: Deathmatch,0,0.99,4.99,80,2004,7588,878,...,72,4,289,Action,"{'Action': 854, 'FPS': 473, 'Multiplayer': 395...",0,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}",English
340,0,2.519678,Half-Life 2: Lost Coast,0,0.00,0.00,0,2005,6986,1121,...,29,273,28,Action,"{'FPS': 1005, 'Action': 346, 'Singleplayer': 2...",0,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}","English, French, German, Italian, Spanish - Sp..."
240,0,2.444157,Counter-Strike: Source,0,1.99,9.99,80,2004,103365,4290,...,408,61,6471,Action,"{'Shooter': 5169, 'Action': 3101, 'FPS': 3055,...",147,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}","English, French, German, Italian, Japanese, Ko..."
72850,1,2.362412,The Elder Scrolls V: Skyrim,15,19.99,19.99,0,2011,269555,15342,...,2708,239,9180,RPG,"{'Open World': 9798, 'RPG': 8514, 'Fantasy': 6...",75,['bethesda game studios'],['bethesda softworks'],"{'windows': True, 'mac': False, 'linux': False}","English, French, German, Italian, Spanish - Sp..."
210770,0,2.280446,Sanctum 2,0,3.74,14.99,75,2013,12092,1335,...,206,0,79,"Action, Indie, Strategy","{'Tower Defense': 809, 'FPS': 554, 'Co-op': 47...",50,['Other'],['coffee stain publishing'],"{'windows': True, 'mac': True, 'linux': True}","English, German, French, Italian, Spanish - Sp..."
8930,0,2.272835,Sid Meier's Civilization® V,0,29.99,29.99,0,2010,157185,6360,...,2736,609,23500,Strategy,"{'Turn-Based Strategy': 4818, 'Strategy': 4208...",286,"['firaxis games', 'aspyr (mac)', 'aspyr (linux)']","['2k', 'aspyr (mac)', 'aspyr (linux)']","{'windows': True, 'mac': True, 'linux': True}","English, French, German, Italian, Spanish - Sp..."
212680,1,2.270903,FTL: Faster Than Light,0,2.49,9.99,75,2012,52003,2054,...,517,124,1739,"Indie, Simulation, Strategy","{'Rogue-like': 2695, 'Space': 2461, 'Strategy'...",51,['Other'],['Other'],"{'windows': True, 'mac': True, 'linux': True}","English, French, Italian, German, Spanish - Sp..."
400,1,2.201865,Portal,0,1.99,9.99,80,2007,70435,1285,...,183,297,1260,Action,"{'Puzzle': 3803, 'Puzzle-Platformer': 2884, '3...",15,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}","English, French, German, Russian, Danish, Dutc..."
620,1,2.201316,Portal 2,0,1.99,9.99,80,2011,197861,2485,...,498,204,6674,"Action, Adventure","{'Platformer': 6869, 'Dark Humor': 6396, 'Puzz...",51,['valve'],['valve'],"{'windows': True, 'mac': True, 'linux': True}","English, French, German, Spanish - Spain, Czec..."
